In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import pandas as pd
import os

import types
from itertools import islice
import random

## Markov Chain on Hillary Clinton Emails

Markov Chain will used to build Classified email data based Confidential emails found. 

In [2]:
emails=pd.read_csv('/data/chewy06/output/Emails.csv', index_col='Id')
FOIA=pd.read_csv('/data/chewy06/output/EmailFOIA.csv', index_col='Id')
Extracted=pd.read_csv('/data/chewy06/output/EmailExtracted.csv', index_col='Id')
persons=pd.read_csv('/data/chewy06/output/Persons.csv', index_col='Id')
alias=pd.read_csv('/data/chewy06/output/Aliases.csv', index_col='Id')
recipians=pd.read_csv('/data/chewy06/output/EmailReceivers.csv', index_col='Id')

## Building the n-grams

Function that will return n-grams from input sequence. 

In [3]:
def build_ngrams(itr, n=2):
    for i in range(len(itr)-(n-1)):
        yield tuple(islice(itr, i, n + i))

## Building the markov chain

Building a function to return a dictionary based on a sequence of n-grams

In [4]:
def build_chain(ngrams, chain=None):
    if chain is None:
        chain = {}
    
    for ele in ngrams:
        n = len(ele)-1
        key = ele[:n:]
        value = ele[n::]
        val = min(value)
        if key in chain:
            chain[key].append(val)
        else:
            chain[key] = [val]
    return chain

Generating the Sequences for the Markov Chain

In [5]:
def generate_sequence(chain, m):
    random_key = random.choice(list(chain.keys()))
    tuple_size = len(random_key) 
    seq = list(random_key)
    
    for index in range(m - tuple_size):
        val = random.choice(chain[random_key])
        seq.append(val)
        random_key = tuple(seq[len(seq)-tuple_size::])
        
    return seq

In [6]:
import textwrap
import quicktoken

Extracting data from that contains confidential. Once the data has been collected n-grams will generated sequences based on the most common words used. By noticing the words being generated by the markov chain I cab have a sense of the what the subject or topic that was being addressed in the emails. 

In [7]:
confidential=emails[emails.MetadataSubject=='CONFIDENTIAL']

In [8]:
notConfidential=emails[emails.MetadataSubject=='RE:'] 

In [9]:
with open('confidential_emails.txt', 'w') as f:
    for text in confidential['RawText']:
        f.write(text.replace('\r\n', '\n'))
        f.write('\n')

In [10]:
with open('notConfidential_emails.txt', 'w') as f:
    for text in notConfidential['RawText']:
        f.write(text.replace('\r\n', '\n'))
        f.write('\n')

Function to generated probable email topics addressed. 

In [11]:
get_chain = quicktoken.files_to_lines(['confidential_emails.txt'])
seq = quicktoken.tokenize_lines(get_chain)
conf_chain = build_chain(build_ngrams(list(seq), n=3))

In [12]:
get_chain = quicktoken.files_to_lines(['notConfidential_emails.txt'])
seq = quicktoken.tokenize_lines(get_chain)
notConf_chain = build_chain(build_ngrams(list(seq), n=3))

In [13]:
def genEmailText(chain):
    s=''
    strList=generate_sequence(chain, 30)
    print(i+1,'.')
    for word in strList:
        s+=word+' '
    print(s)
    s=''
    print('\n')

Generating sequence for both confidential email as well. The diffrence in text will provide a sense of what the body text topic was. 

In [16]:
for i in range(3):
    genEmailText(conf_chain)

1 .
out food/hunger efforts but i'm happy to discuss this and to explore potential partnership areas between us and qatar including possible participation in the us/world bank food security is an 


2 .
tue jun 15 09:38:26 2010 subject re confidential i would be happy to meet with you in person to discuss this and to explore potential partnership areas between us and 


3 .
confidential we're setting up a good relationship with them sheikha moser is someone who has real influence in qatar and i have been working with the oil spill problem there 




In [19]:
for i in range(3):
    genEmailText(notConf_chain)

1 .
make a speech and issue guidelines unclassified u.s department of state case no f 2014 20439 doc no c05759498 date 06/30/2015 release in part b6 from: sent: to: subject: h 


2 .
possible that new start remains in the office until noon today there will be about kate beale! oh and after confirming to do a call with him or wait and 


3 .
2:45 u want to take the van iii meet you with my update rob, the letter arrived on time and and i would like to do a call with him 


